### Step 1: Edge detection with canny
Lý do: Ta cần phát hiện cạnh để tách biệt các vật thể + loại bỏ nhiễu, giúp cho việc nhận diện dễ dàng hơn

In [79]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import imutils
import cv2

orig = cv2.imread("Image/test_01.jpg")
image = orig.copy()
image = imutils.resize(image, height=1500)

In [80]:
# gray it
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# blur it
gray_blur = cv2.GaussianBlur(gray, (5, 5), 0)
# canny edge detection
# Có Sobel và Prewitt nữa nhưng mà chúng nó nhạy cảm hơn với nhiễu + xử lý tương phản thấp ko tốt
# Canny threshold: nếu pixel < low thì bị loại bỏ, > high chắc chắn được giữ, nếu nằm giữa thì xem xét nó có liên kết với một strong edge không
edged = cv2.Canny(gray_blur, 20, 50)

cv2.imshow("canny", imutils.resize(edged, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

### Step 2: Apply a perspective transform to extract the top-down, birds-eye-view of the exam.

#### 2.1: Find contour of answer sheet

In [81]:
contours_outer_edge = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_outer_edge = imutils.grab_contours(contours_outer_edge)
sheetCnts = None
count = 0
if len(contours_outer_edge) > 0:
    contours_outer_edge = sorted(contours_outer_edge, key=cv2.contourArea, reverse=True)
    for c in contours_outer_edge:
        # calc the perimeter
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.05*peri, True)
        if len(approx) == 4:
            count += 1
            sheetCnts = approx
            cv2.drawContours(image, [sheetCnts], -1, (0, 255, 0), 2)
            cv2.imshow("point", image)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
            break

#### 2.2 Apply four_point_transform to display bird's eyes view (top-down)

In [82]:
paper = four_point_transform(image, sheetCnts.reshape(4, 2))
warped = four_point_transform(gray_blur, sheetCnts.reshape(4,2))
cv2.imshow("bird", paper)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Step 3: Extract the set of bubbles (i.e., the possible answer choices) from the perspective transformed exam.

In [83]:
thresh = cv2.adaptiveThreshold(
    warped, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
cv2.imshow("thresh", imutils.resize(thresh, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [84]:
# vẽ viền cho tất cả để xác nhận lại viền có đúng không
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
print(len(cnts))

cv2.drawContours(warped, cnts, -1, (0, 255, 0), 1)
cv2.imshow('thres', imutils.resize(warped, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

837


In [85]:
show_boxes = True
questions = []
print(len(cnts))
for c in cnts:
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    if (w >= 15 and h >= 15) and (w <= 50 and h <= 50) and ar >= 0.7 and ar <= 1.3:
        questions.append(c)
print(len(questions))


cv2.drawContours(paper, questions, -1, (255, 0, 0), 1)
cv2.imshow('questions', imutils.resize(paper, height=700))
cv2.waitKey(0)
cv2.destroyAllWindows()

837
240


In [86]:
# Answer Key
ques = [i for i in range(60)]
opts = [0 for _ in range(60)]
ANSWER_KEY = dict(zip(ques, opts))

In [87]:
# Get question contours and order
questionCnts = contours.sort_contours(questions, method="top-to-bottom")[0]

questionCnts_ordered = []
for i in np.arange(0, len(questionCnts), 16):
        ordered = contours.sort_contours(questionCnts[i:i+16])[0]
        for q in ordered:
             questionCnts_ordered.append(q)

bubble_thresh = 185
correct = 0
# Lặp qua các contours của từng câu hỏi
for (q, i) in enumerate(np.arange(0, len(questionCnts_ordered), 4)):
    # Sắp xếp các contours cho câu hỏi hiện tại từ trái sang phải, sau đó khởi tạo chỉ số của câu trả lời đã được chọn
    cnts = contours.sort_contours(questionCnts_ordered[i:i + 4])[0]
    bubbled = [-1, -1]
    bubble_count = 0

    # Lặp qua các contours đã được sắp xếp
    for (j, c) in enumerate(cnts):
        # Tạo mask để chỉ hiển thị "bubbles" cho câu hỏi hiện tại
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        # Áp dụng mask vào ảnh threshold, sau đó đếm số lượng pixel khác không trong vùng bubble
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)

        # if total > current bubbled then
        # bubbled = total
        if bubbled[0] == -1 or bubbled[0] < total:
            bubbled = (total, j)
            if bubbled[0] > bubble_thresh:
                bubble_count += 1
        else:
            # update bubble thresh
            # if total <= bubbled[0]
            # this means that bubbled is not filled
            # therefore we are updating bubble thresh to adjust to the pic
            bubble_thresh = max(bubble_thresh, total+5)

    # Khởi tạo màu của contour và chỉ số của đáp án
    color = (0, 0, 255)
    k = ANSWER_KEY[q]

    # Kiểm tra xem câu trả lời đã được chọn có đúng không
    if k == bubbled[1] and bubbled[0] > bubble_thresh:
        color = (0, 255, 0)
        correct += 1
    # Vẽ viền của câu trả lời đúng trên bài kiểm tra
    cv2.drawContours(paper, [cnts[k]], -1, color, 3)
print(correct)
cv2.imshow('show answer', paper)
cv2.waitKey(0)
cv2.destroyAllWindows()

2
